This notebook details the **ETL pipeline** used in the project.

Both the data source and destination refer to the same Google Sheets [here](https://docs.google.com/spreadsheets/d/18KGQFVDZ8wgj2pb1ErKh2QXGtF1N2U17GiwRBcXI1TI/). 

Summary:
- Extract
    - VRN data from the "VRN" worksheet
    - Car prices data from the "Prices" worksheet
- Transform
- Load
    - Results data into the "Results" worksheet
    - Updated car prices data into the "Prices" worksheet

In [7]:
# necessary for pyspark to be correctly loaded
import findspark
findspark.init()

import os
while not os.getcwd().endswith('vrn-analysis'):
    os.chdir('..')

# ETL pipeline

The main job that runs the ETL pipeline is the `main()` function in `jobs/etl_job.py`.

`start_spark()` is a custom function defined in the `helpers` package that returns the created Spark session, Spark logger and config values in a dict.

In [2]:
from helpers.spark import start_spark

# start Spark application and get session, logger, config
spark, log, config = start_spark(
    app_name='vrn_analysis',
    files=['configs/etl_config.json'])
sc = spark.sparkContext

## Extract

#### VRN worksheet

The **VRN** worksheet serves as the central data repository that stores the car types for the various VRN letter/number combinations.

Each row represents one VRN letter, e.g. *SMS*. <br>
Each column represents one number, e.g. *1*.

A snippet of a row in the "VRN" worksheet is shown below.

<img src="img/vrn-ws-sample.jpg" alt="VRN WS sample" />

The data for each cell here had been painstakingly collected, one by one, from the LTA road tax expiry enquiry site [here](https://vrl.lta.gov.sg/lta/vrl/action/pubfunc2?ID=EnquireRoadTaxExpDtProxy). Sadly, there is no convenient API where this information can be extracted.

The cell text and background colour varies depending on the car brand to make for better visual aesthetics.

The image below lists all of the car brands and their corresponding colours.

<img src="img/car-brands.jpg" alt="Car brands"/>

*Side note: "FERRARI" requires an extra " /" behind it as there is a model under FIAT called "FIAT / 500 ABARTH 695 TRIBUTO FERRARI 1.4 A" that would be mistakenly coloured as under the FERRARI brand otherwise.*

#### Prices worksheet

The **Prices** worksheet lists all of the car types present and the corresponding price for each car type.

It contains 3 columns - make, model, price.

<img src="img/prices-ws-sample.jpg" alt="Prices WS sample" />

In [8]:
from jobs import extract

# Extract phase: get VRN + Prices sheets as RDDs
vrn_rdd, prices_rdd = extract.run(
    sc, 
    log,
    config)

print('VRN worksheet:')
print(*vrn_rdd.take(2), sep='\n')
print('\nPrices worksheet:')
print(*prices_rdd.take(3), sep='\n')

VRN worksheet:
['Series', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '18', '28', '88', '99', '100', '888', '999', '8888']
['SKF', 'ROLLS ROYCE / PHANTOM AUTO ABS D/AB 2WD 4DR HID TV NAV', 'MAZDA / ROADSTER 1.5 AUTO', 'LAMBORGHINI / AVENTADOR LP700-4', 'MERCEDES BENZ / 280S', 'PORSCHE / PANAMERA G2 PDK E6', 'SUZUKI / SWIFT HYBRID 1.2RS CVT', 'FERRARI / 458 SPIDER 4.5L A/T ABS D/AB 2WD 2DR HID', 'PORSCHE / 911 CARRERA (991-II) PDK E6 SES', 'TOYOTA / ALPHARD 2.5SC PACKAGE CVT ABS 2WD 5DR', '-', 'B.M.W. / X6 XDRIVE35I SR LED NAV HUD', 'B.M.W. / 520I LED NAV', 'VOLKSWAGEN / GOLF TSI TL', 'MERCEDES BENZ / GLC250 4MATIC (R18 LED)', 'TOYOTA / LEXUS RX350', 'BENTLEY / CONTI FS', 'NISSAN / GT-R 3.8 A', 'MERCEDES BENZ / V250 D AVG LONG']

Prices worksheet:
['ALFA ROMEO', 'GIULIA 2.9 V6', '339,000']
['ALPINA', 'B3 BITURBO', '318,800']
['ALPINE', 'A110', '238,800']


Each worksheet is loaded into an **RDD as a list of lists**, where each sublist contains the contents of a single row.

## Transform

### `vrn_rdd`

`vrn_rdd` will be transformed to generate 2 output RDDs:

- `vrn_rdd_tfm`, which contains the cleaned version of the car model names
- `results_rdd`, which contains the price mapping for each car model

##### Cleaning of car type values

The cleaning refers to the removal of extra descriptors in the car model name that serves no purpose, as well as converting similar-sounding model names into one common model name to prevent duplicates. **This portion forms the meat of the entire transformation process.**

Example of removing extra descriptors:

- From:
    - *B.M.W. / M5 30 JAHRE EDITION SR NAV NVD LED HUD*
    - *MERCEDES BENZ / A45 AMG AT 4MATIC D/AIRBAG 2WD 5DR*
- To
    - *B.M.W. / M5 30 JAHRE EDITION*
    - *MERCEDES BENZ / A45 AMG 4MATIC*

Example of converting to a common model name:

- From:
    - *TOYOTA / ALPHARD 2.5SC CVT*
    - *TOYOTA / ALPHARD 7-SEATER 2.5 SC CVT*
    - *TOYOTA / ALPHARD 2.5S-C CVT*
- To:
    - *TOYOTA / ALPHARD 2.5 S-C*
    
**`vrn_rdd_tfm` will be the output of this subprocess.**

##### Mapping each car type to a price

This car-type-to-price mapping is based on a car prices dict, which is created from `prices_rdd`. A sample is as follows.

```
{
    'ALPINA / B3 BITURBO': '318,800',
    'ASTON MARTIN / DB11 V8': '799,000',
    'ASTON MARTIN / DB9': '808,000',
    'ASTON MARTIN / RAPIDE 6.0': '868,000',
    'ASTON MARTIN / V8 VANTAGE S/SHIFT 4.7': '520,000',
    'AUDI / A3 SEDAN 1.0': '136,460',
    'AUDI / A3 SPORTBACK 1.0': '134,250',
    ...
}
```

After the requisite cleaning is done, the price for each car type can then be obtained via a simple dict lookup.
    
**`results_rdd` will be the output of this subprocess.**

### `prices_rdd`

`prices_rdd` is not so much transformed, but updated with new make/model combinations that were not in the original list, and hence no corresponding price could be found for these new combinations.

During the mapping phase above, any car type that did not have a corresponding price will be flagged out.

These car types are then added to `prices_rdd`, where they will be assigned a default value of `0`.

Example of new car type entry:

```
    [
     ...,
     ['BENTLEY', 'CONTINENTAL FLYING SPUR 6.0', '0'],
     ...,
    ]
```

**`prices_rdd` will be the output of this subprocess.**

#### (Additional note)

Of course, this entire process is a chicken-and-egg cycle, where the car prices have to be found first before a mapping can be done, but the car types have to be first identified before the corresponding car prices can be found.

This transformation process only works after a base has been established. The **Laying the foundations** section below details the effort that went into building this base, which contains additional details on how the cleaning of car type values was initially done.

In [9]:
from jobs import transform

# Transform phase:
# convert each car model to a price
# update prices RDD with any new car types 
vrn_rdd_tfm, results_rdd, prices_rdd_tfm = transform.run(
    log,
    vrn_rdd,
    prices_rdd)

print('Transformed VRN worksheet:')
print(vrn_rdd_tfm.take(40))

Transformed VRN worksheet:
['Series', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '18', '28', '88', '99', '100', '888', '999', '8888', 'SKF', 'ROLLS ROYCE / PHANTOM', 'MAZDA / ROADSTER 1.5', 'LAMBORGHINI / AVENTADOR LP700-4', 'MERCEDES BENZ / 280S', 'PORSCHE / PANAMERA', 'SUZUKI / SWIFT HYBRID 1.2', 'FERRARI / 458 SPIDER 4.5', 'PORSCHE / 911 CARRERA', 'TOYOTA / ALPHARD 2.5 S-C', '-', 'B.M.W. / X6 XDRIVE35I', 'B.M.W. / 520I', 'VOLKSWAGEN / GOLF 1.4', 'MERCEDES BENZ / GLC250 4MATIC', 'TOYOTA / LEXUS RX350', 'BENTLEY / CONTINENTAL FLYING SPUR', 'NISSAN / GT-R 3.8', 'MERCEDES BENZ / V250', 'SKG', 'ROLLS ROYCE / GHOST EWB']


In the VRN worksheet, the cell values which contained a car type are now replaced with a price.

For car types where no price mapping was found, they are added as new rows to the Prices worksheet.

In [10]:
print('New values in Prices worksheet:')
new_values = prices_rdd_tfm.filter(lambda x: x[2] == '0').take(100)
print(*new_values, sep='\n')

New values in Prices worksheet:
['FERRARI', '360 F1 SPIDER', '0']
['FERRARI', '430 F1 SPIDER', '0']
['FERRARI', 'FERRARI 599F1', '0']
['MASERATI', 'GRANTURISMO CAMBIOCORSA', '0']
['MASERATI', 'GRANTURISMO MC STRADALE', '0']
['MITSUBISHI', 'EVO IV', '0']
['PORSCHE', '981 BOXSTER S', '0']
['ROLLS ROYCE', 'SILVER SPUR', '0']


## Load

### `vrn_rdd_tfm`

The transformed VRN RDD contains the cleaned version of the car model names.

<img src="img/vrncleaned-ws-sample.jpg" alt="VRNCleaned WS sample" />

This will be uploaded to a **VRNCleaned** worksheet.

### `results_rdd`

The results RDD follows the same structure as the VRN RDD, where each cell contains the price of the corresponding car type.

<img src="img/results-ws-sample.jpg" alt="Results WS sample" />

This RDD will be uploaded to a **Results** worksheet.

### `prices_rdd`

The prices RDD has been updated with new make/model combinations and will be uploaded back to its source, the **Prices** worksheet.

In [11]:
from jobs import load

# Load phase: Load the transformed RDDs as CSVs and upload back to GSheets
n_cols = len(vrn_rdd.take(1)[0])
load.run(
    log,
    config,
    n_cols,
    vrn_rdd_tfm,
    results_rdd,
    prices_rdd_tfm)

In the **Prices** worksheet, values that contain the value "0", i.e. new car types, are highlighted in red for easier identification. Manual intervention will then be required to find the corresponding price for these new car types.

# Laying the foundations

This section details what was done to lay the initial foundations for this project, before the ETL pipeline described above could be established.

This can be broken down into 3 sections:
1. Data collection
1. Car type cleaning
1. Finding car prices

### Data collection

The data collection process comprises of 2 steps.

#### Calculating the VRN checksum letters

This refers to finding the checksum letter, or the ending letter, in any licence plate. This calculation process can be found online and has been translated into Python code in the file `scripts/vrn_checksum.py`.

To add new letters/numbers, simply add to the 2 lists - `VRN_LETTERS`, `VRN_NUMBERS` - defined at the top of the file and call `vrn_checksum.run()`.

This will calculate the checksum for all of the letter/number combinations and automatically upload the results to the **Checksum** worksheet. A sample is shown below.

<img src="img/checksum-ws-sample.jpg" alt="Checksum WS sample" />

#### Actual data collection

The **Checksum** worksheet can then be used as reference for input into the LTA road tax expiry enquiry site [here](https://vrl.lta.gov.sg/lta/vrl/action/pubfunc2?ID=EnquireRoadTaxExpDtProxy).

1. Input the vehicle number and fill in the CAPTCHA.

    <img src="img/data-collection-sample-1.jpg" alt="Data collection 1" width="80%" />

1. Copy the entire text from the "Vehicle Make/Model" field into Google Sheets.

    <img src="img/data-collection-sample-2.jpg" alt="Data collection 2" width="80%" />

### Car type cleaning

The most time-consuming part of the entire process is to clean the myriad of make/model combinations. The details are mentioned above in the **Transform** section.

We will take 1 brand as an example and do a walkthrough on how the specific cleaning is done.

First, based on `vrn_rdd`, we create a dict `cars_dict` where:
- key: car make
- value: list of car models belonging to the car make

In [ ]:
from jobs.transform_scripts import cleaners, general
cars_dict = vrn_rdd \
    .flatMap(lambda x: x) \
    .filter(lambda x: general.is_car_type(x)) \
    .map(lambda x: cleaners.clean_raw_data(x)) \
    .map(lambda x: x.split(' / ')) \
    .reduceByKey(lambda x, y: x + ',' + y) \
    .map(lambda x: (x[0], x[1].split(','))) \
    .map(lambda x: (x[0], sorted(set(x[1])))) \
    .sortByKey(lambda x: x[0]) \
    .collectAsMap()

Then, we focus on each make at a time. Let's take "*BENTLEY*" as an example.

In [ ]:
import re

def clean(model_name):
    model_name = re.sub('TOYOTA / TOYOTA /', 'TOYOTA /', model_name)
    model_name = re.sub('ESTIMA', 'PREVIA', model_name)

    # remove the following words
    model_name = re.sub('\s(7(\s|-)SEATER|\dDR|\dWD|ABS|A\/?T|(D\/)?AIRBAG|AUTO|CVT|EDITION|EXECUTIVE\sLOUNGE|G\'S|M(OON)?R(OOF)?|PACKAGE|PLATINUM|PREMIUM|S\/R|SEDAN|SELECTION|ST(ANDAR)?D|SUV)', '', model_name)
    model_name = re.sub('^TOYOTA ', '', model_name)
    model_name = re.sub('-PACKAGE', '', model_name)
    
    return model_name


[clean(e) for e in cars_dict['TOYOTA']]

First, we remove the unnecessary words.
- These can be obvious unnecessary words, e.g. *4WD*, *5 SEATER*, *ABS*, *AT*, *AUTO*, *S/R*
- Reference to pricelists also help to decide if a word can be removed, e.g. if there only exists *BENTAYGA V8* in the pricelist, then the word *DIESEL* can be removed

For efficient cleaning, we use regex patterns - https://regexr.com/ is an invaluable tool here for us to test out our regex patterns.

<img src="img/data-cleaning-sample.jpg" alt="Data cleaning sample" width="80%" />

In [ ]:
import re

def clean(s):
    return re.sub('\s(\d\sSEATER|ABS|A\/?T|A(UTO)?|DIESEL|\dWD|S\/R)', '', s)
    
sorted(list(set([clean(s) for s in cars_dict['BENTLEY']])))

After the extra words have been removed, we manually examine the remaining words more closely.

We see that there are some models that should be combined into the same model type.
- *CONTINENTAL GT V8*, *CONTINENTAL GT V8 S*
- *FLYING SPUR V8 4.0*, *FLYING SPUR V8 S*

We add some manual substitutions to deduplicate these similar model names.

In [ ]:
def clean(s):
    s = re.sub('SPUR V8 S', 'SPUR V8 4.0', s)
    s = re.sub('GT V8 S', 'GT V8', s)
    return re.sub('\s(\d\sSEATER|ABS|A\/?T|A(UTO)?|DIESEL|\dWD|S\/R)', '', s)
    
sorted(list(set([clean(s) for s in cars_dict['BENTLEY']])))

Now, we are satisfied that no further cleaning can be done.

The cleaning function can then be copied to the `jobs/transform_scripts/cleaners.py` file. For easier maintanability, the cleaning function for each car make is located in its own function. In this case, the function should be renamed to `clean_bentley()`.

The above process is then repeated for each car make. A tedious process...

### Finding car prices

Finding car prices can be a straightforward or tiresome task, depending on a few factors:
- Car make
    - Most car brands have published pricelists
    - But super high-end brands, e.g. *BENTLEY*, *FERRARI*, typically label all their car prices as "POA" (Price On Application)
    - And there are some brands that I never knew existed and where an accurate price is almost impossible to find, e.g. *RUF*
- Car age
    - It's easy to find the price for a *MERCEDES BENZ / S400L*, but how can we find an accurate market price for a *MERCEDES BENZ / 300SL-24* that was last produced in 2001?
- Car model rarity
    - There exists a *B.M.W. / M5*, which is a high-performance BMW saloon
    - There also exists a *B.M.W. / M5 30 JAHRE EDITION*, which is a special edition of the M5 built to commemorate the 30-year anniversary of the original BMW M5 release. Only 300 of such vehicles were produced - how can this collector's item be accurately quantified?
- Car model variations
    - The following models all exist:
        - *MERCEDES BENZ / GLC250 4MATIC*
        - *MERCEDES BENZ / GLC250 4MATIC COUPE*
        - *MERCEDES BENZ / GLC250 AMG 4MATIC*
        - *MERCEDES BENZ / GLC250 AMG 4MATIC COUPE*
    - Sometimes, I just try to find the price for one of these models and then approximate the prices for the others using the found price as the baseline, e.g. add \\$10k for AMG, add $20k for coupe, etc.

Additionally, some car models are likely to be an older model despite sharing the same model name, e.g. *MERCEDES BENZ / C180 KOMP*, *MERCEDES BENZ / C180 CGI*. However, for simplicity's sake, the model years are all ignored and all of the variations are assigned the current price of the latest model.

Another factor is COE - some of the prices include a COE package, while others exclude it. For simplicity's sake as well, all of those are disregarded. It is too much of a pain to sieve through the fine print on each pricelist...

**These are the 2 main resources that I had used to find the car prices:**
- [sgCarMart](https://www.sgcarmart.com/new_cars/newcars_pricelist_listing.php)
- [OneShift](https://www.oneshift.com/new_cars/car-price-singapore)

When all else fails, typing "*{car model name} price singapore*" into Google Search sometimes works pretty well.

As the GSheet document is taken as the SSoT, **all prices found should be updated in GSheets directly**.

<img src="img/prices-ws-sample.jpg" alt="Prices WS sample" />

After the prices have been updated in GSheets, the following cycle can then start (or continue):

<img src="img/process-cycle.jpg" alt="Process cycle" />